# Face detection Using OpenCV

OpenCV deep learning face detector is based on Single-Shot_Detector (SSD) framework with a ResNet base network (unlike other OpenCV SSDs that you may have seen which typically use MobilNet as the base network). 

In first part we will see how to apply face detection to single input images. 

Then in next section, we will check how to modify this code and apply face detection to videos, video streams and webcams. 

## Part 1: Face detection on images

In [2]:
# import require libraires
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib inline

In [35]:
# input arguments
# input image 
#arg_image = ".//DATA//images//rooster.jpg"
arg_image = ".//DATA//images//iron_chic.jpg"
# path to Caffe deploy prototxt file
arg_prototxt = ".//MODELS//FaceDetection//deploy.prototxt"
# path to Caffe pre-trained model
arg_model = ".//MODELS//FaceDetection//res10_300x300_ssd_iter_140000.caffemodel"
# minimum probability to filter weak detections
arg_confidence = 0.5

In [36]:
# load the mode
print("[INFO] Loading face detection model...")
net = cv2.dnn.readNetFromCaffe(arg_prototxt, arg_model)

# load input image
image = cv2.imread(arg_image)
# capture its shape
(h,w) = image.shape[:2]
# convert image to blob to feed to net
blob = cv2.dnn.blobFromImage( cv2.resize(image,(300,300)),1.0, (300,300), (104.0, 177.0, 123.0))


[INFO] Loading face detection model...


In [37]:
# computing face detection
net.setInput(blob)
# get predictions
detections = net.forward()

In [38]:
# detections is array of 200 element where each element is of 7 values
# these 7 values are: 
# (x,class_label(face),confidence,startX,startY,endX,endY)
# last 4 are BB coordinates
print(detections.shape)
detections

(1, 1, 200, 7)


array([[[[0.        , 1.        , 0.9999083 , ..., 0.11946487,
          0.86777306, 0.5585621 ],
         [0.        , 1.        , 0.9985989 , ..., 0.25462836,
          0.52284336, 0.6467794 ],
         [0.        , 1.        , 0.99735343, ..., 0.30858952,
          0.2786947 , 0.7022801 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

Lets toop over detections and draw bounding box

In [39]:
# loop over the detections
for i in np.arange(0,detections.shape[2]):
    # extract the confidence (i.e. probability)
    # of finding a face
    confidence = detections[0,0,i,2]
    # filter out weak detections by checking confidence
    # with minimum confidence
    if(confidence > arg_confidence):
        print("confidence: ",confidence)
        # get BB cordinates
        # compute the (x,y) cordinates of BB of the object
        box = detections[0,0,i,3:7] * np.array([w,h,w,h])
        (startX, startY, endX, endY) = box.astype(int)
        print(startX, startY, endX, endY)
        # draw the bounding box of face along with label+probability
        text = "{:.2f}%".format(confidence*100)
        y = startY -10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(image, (startX, startY), (endX, endY),(0,0,255),2)
        cv2.putText(image, text, (startX,y),cv2.FONT_HERSHEY_SIMPLEX,0.45,(0,0,255),2)

# displat the final image
cv2.imshow("Face detected", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


confidence:  0.9999083
260 59 433 279
confidence:  0.9985989
135 127 261 323
confidence:  0.99735343
6 154 139 351


## Part 2: Face detection on videos / webcam

In [1]:
# import required libraries
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2

In [2]:
# define input arguments
# path to Caffe deploy prototxt file
arg_prototxt = ".//MODELS//FaceDetection//deploy.prototxt"
# path to Caffe pre-trained model
arg_model = ".//MODELS//FaceDetection//res10_300x300_ssd_iter_140000.caffemodel"
# minimum probability to filter weak detections
arg_confidence = 0.5

In [10]:
# load the model
print("[INFO] loading face detection model...")
net = cv2.dnn.readNetFromCaffe(arg_prototxt, arg_model)
# initialize the video stream
print("[INFO] starting video stream...")
# use below for webcam
vs = VideoStream(src=0).start()
# for Raspberry Pi + picamera use below
#vs = VideoStream(usePiCamera=True).start()
# for video file swap out the VideoStream calss for FileVideoStream

# allow camera sensor to warm up for 2sec
time.sleep(2.0)

# loop over the frames and compute face detections
while True:
    # grab the frame from the threaded video stream and resize it 
    # to have a maximum width of 400 pixels
    frame = vs.read()
    #frame = imutils.resize(frame, width=600)
    frame = imutils.resize(frame, width=750, height=500)
    # grab the frame dimension and conver it to blob
    (h,w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage( cv2.resize(frame,(300,300)), 1.0,
                                (300,300),(104.0,177.0,123.0))
    
    # pass the blob thru the network and obtain 
    # the detection and predictions
    net.setInput(blob)
    detections = net.forward()
    # detection has the same structure as describe above
    # while working with images
    
    # loop over the detection
    for i in np.arange(0, detections.shape[2]):
        # extract the confidence
        confidence = detections[0,0,i,2]
        # filter out the weak detections as per minimum confidence
        if confidence < arg_confidence:
            continue
        
        # compute the (x,y) cordinates of BB for the face detected
        box = detections[0,0,i,3:7] * np.array([w,h,w,h])
        (startX, startY, endX, endY) = box.astype("int")
        # draw the BB of the face along with the associated probability
        text = "{:.2f}%".format(confidence*100)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(frame, (startX, startY), (endX, endY), 
                     (0,0,255),2)
        cv2.putText(frame, text, (startX,y), cv2.FONT_HERSHEY_SIMPLEX,
                   0.45,(0,0,255),2)
        
    # show the output frame
    cv2.imshow("Video-Frame",frame)
    #key = cv2.waitKey(1) & 0xFF
    # if the "q" key was pressed, break from the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
# release handle to the webcam and clean up
vs.stop()
cv2.destroyAllWindows()    

[INFO] loading face detection model...
[INFO] starting video stream...
